In [1]:
%matplotlib inline
import torch
import numpy as np
import torchvision
from torch.utils import data
from torchvision import transforms


In [2]:
# 通过ToTensor实例将图像数据从PIL类型变换成32位浮点数格式，
# 并除以255使得所有像素的数值均在0到1之间
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(
root="../data", train=False, transform=trans, download=False)

In [3]:
#数据读取
batch_size = 256
def get_dataloader_workers(): 
#"""使⽤4个进程来读取数据"""
    return 4
train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True,
num_workers=get_dataloader_workers())
test_iter = data.DataLoader(mnist_test, batch_size, shuffle=True,
num_workers=get_dataloader_workers())

mnist_train[0][0].shape


torch.Size([1, 28, 28])

在这里，我们可以看到，每个数据都是 1 * 28 * 28 的形式，我们在这里，先将数据展平，每个数据有 28 * 28=784 个特征值。而输出共有10个特征。我们现在设计一层网络，即：输入-输出 网络，则权重为 784  * 10偏重为10 * 1，最终得到1 * 10的向量

In [4]:
def accuracy(y_hat, y): #@save
#"""计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [5]:
from torch import nn
net=nn.Sequential(nn.Linear(784,10))
w=net[0].weight.data.normal_(0, 0.01)
b=net[0].bias.data.fill_(0)
loss=nn.CrossEntropyLoss()
sgd=torch.optim.SGD(net.parameters(), lr=0.03)


In [6]:
num_epochs=30
for epoch in range(num_epochs):
    a=[]
    for X,y in train_iter:
        y_hat=net(X.reshape((-1, w.shape[1])))
        l=loss(y_hat,y)
        sgd.zero_grad()
        l.backward()
        sgd.step()
        ac=accuracy(y_hat,y)/len(y)
        a.append(ac)
    print(f'epoch {epoch + 1}, loss 从{a[0]}到{a[-1]}')

C:\Users\11215\anaconda3\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:112.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


epoch 1, loss 从0.1640625到0.8229166666666666
epoch 2, loss 从0.75到0.7708333333333334
epoch 3, loss 从0.82421875到0.8333333333333334
epoch 4, loss 从0.80078125到0.84375
epoch 5, loss 从0.828125到0.8229166666666666
epoch 6, loss 从0.79296875到0.8333333333333334
epoch 7, loss 从0.85546875到0.84375
epoch 8, loss 从0.84375到0.8854166666666666
epoch 9, loss 从0.83984375到0.7708333333333334
epoch 10, loss 从0.85546875到0.84375
epoch 11, loss 从0.83984375到0.8333333333333334
epoch 12, loss 从0.828125到0.8333333333333334
epoch 13, loss 从0.84765625到0.8333333333333334
epoch 14, loss 从0.890625到0.8020833333333334
epoch 15, loss 从0.890625到0.8333333333333334
epoch 16, loss 从0.82421875到0.75
epoch 17, loss 从0.7890625到0.8125
epoch 18, loss 从0.8671875到0.84375
epoch 19, loss 从0.86328125到0.8854166666666666
epoch 20, loss 从0.88671875到0.8541666666666666
epoch 21, loss 从0.87890625到0.8125
epoch 22, loss 从0.84375到0.84375
epoch 23, loss 从0.8828125到0.8333333333333334
epoch 24, loss 从0.83203125到0.8229166666666666
epoch 25, loss 从0.7968

In [7]:
class Accumulator: 
#"""在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

def evaluate_accuracy(net, data_iter): 
#"""计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval() # 将模型设置为评估模式
    metric = Accumulator(2) # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X.reshape((-1, w.shape[1]))), y), y.numel())
    return metric[0] / metric[1]

print(evaluate_accuracy(net, test_iter))
print(evaluate_accuracy(net, train_iter))

0.8307
0.8493
